<a href="https://colab.research.google.com/github/shown5/HandsOnLLM/blob/main/chap6_prompt_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -U langchain>=0.1.17 openai>=1.13.3 langchain_openai>=0.1.6 transformers>=4.40.0 datasets>=2.18.0 accelerate>=0.31.0 sentence-transformers>=2.5.1 duckduckgo-search>=5.2.2
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# モデルとトークナイザーを読み込む
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

# パイプラインを作成するs
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=500,
    do_sample=False,
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [3]:
# プロンプトを設定
messages = [
    {"role": "user", "content": "ニワトリについての面白いジョークを作って"}
]

# 出力を生成
output = pipe(messages)
print(output[0]["generated_text"])

AttributeError: 'DynamicCache' object has no attribute 'seen_tokens'

In [ ]:
# プロンプトテンプレートを適用する
prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False)
print(prompt)

In [ ]:
# 高い温度を使用する
output = pipe(messages, do_sample=True, temprature=1)
print(output[0]["generated_text"])

In [ ]:
# 高い top_p を使用する
output = pipe(messages, do_sample=True, top_p=1)
print(output[0]["generated_text"])

In [ ]:
# プロンプトの要素
persona = "あなたは大規模言語モデルの専門家です。複雑な論文をわかりやすくようやくすることに秀でています。\n"

instruction = "与えられた論文の主要な発見を要約してください。\n"

context = "要約は、研究者が論文の最も重要な情報を迅速に理解できるよう、最重要ポイントを抽出した内容であるべきです。\n"

data_format = "手法を箇条書きでまとめ、その後に主要な結果を簡潔にようやくした要約した段落を記載してください。\n"

audience = "素早く把握したい忙しい研究者を対象としています。\n"

tone = "トーンは専門的かつ明確であるべきです。\n"

text = "要約するテキスト"

data = f"要約するテキスト: {text}"

# プロンプト全体。生成された出力に与える影響を確認するために、プロンプトから要素を削除・追加する
query = persona + instruction + context + data_format + audience + tone + data

In [ ]:
# 架空の単語を文中で使う例を１つ示す
one_shot_prompt = [
    {
        "role": "user",
        "content": "「ギガムル」は、日本の楽器の一種です。「ギガムル」を使った例文は次のとおりです："
    },
    {
        "role": "assistant",
        "content": "私は叔父から贈り物としてもらったギガムルを持っています。家で演奏するのが大好きです。"
    },
    {
        "role": "user",
        "content": "「スクリッグ」とは、何かに向かって剣を振るうことを指します。「スクリッグ」を使った例文は次のとおりです："
    }
]
print(tokenizer.apply_chat_template(one_shot_prompt, tokenize=False))

In [ ]:
# 出力を生成
outputs = pipe(one_shot_prompt)
print(outputs[0]["generated_text"])

In [ ]:
# 商品名とスローガンを作成
product_prompt = [
    {"role": "user", "content": "LLMを活用するチャットbotの名前とスローガンを作成してください。"}
]
outputs = pipe(product_prompt)
product_description = outputs[0]["generated_text"]
print(product_description)

In [ ]:
# 商品名とスローガンに基づいて、売り文句を生成
sales_prompt = [
    {"role": "user", "content": "以下の製品について、とても短い売り文句を作成してください。"}
]
outputs = pipe(sales_prompt)
sales_pitch = outputs[0]["generated_text"]
print(sales_pitch)

In [ ]:
# Answering with chain-of-thought
cot_prompt = [
    {"role": "user", "content": "Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?"},
    {"role": "assistant", "content": "Roger started with 5 balls. 2 cans of 3 tennis balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11."},
    {"role": "user", "content": "The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?"}
]

# Generate the output
outputs = pipe(cot_prompt)
print(outputs[0]["generated_text"])

In [ ]:
# Zero-shot CoT
zeroshot_cot_prompt = [
  {"role": "user", "content": "食堂にはリンゴが２３個ありました。２０個をランチに使い、新たに６個購入した場合、りんごは何個あるでしょうか？ステップバイステップで考えましょう。"}
]

# 出力を生成
outputs = pipe(zeroshot_cot_prompt)
print(outputs[0]["generated_text"])

In [ ]:
# Zero-shot tree-of-thought
zeroshot_tot_prompt = [
    {"role": "user", "content": "３人の異なる専門家がこの質問に答える様子を想像してください。各専門家はまず１ステップずつ考えを紙に書き、それを他の専門家と共有します。その後、次のステップに進むという形で進行します。途中で誰かが自分の間違いに気づいた場合、その場を離れます。質問は「食堂にはもともと２３個のリンゴがありました。昼食に２０個使って、さらに６個書いました。今、りんごはいくつあるでしょう？」です。結果について必ず議論してください。"}
]

# 出力を生成
outputs = pipe(zeroshot_tot_prompt)
print(outputs[0]["generated_text"])